In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('data_crawl.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39561 entries, 0 to 39560
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Time           39561 non-null  object 
 1   AQI            39561 non-null  float64
 2   PM2.5          39561 non-null  float64
 3   PM10           39561 non-null  float64
 4   CO             39561 non-null  float64
 5   NO2            39561 non-null  float64
 6   O3             39561 non-null  float64
 7   SO2            39561 non-null  float64
 8   NH3            39561 non-null  float64
 9   Date           39561 non-null  object 
 10  Temp           26800 non-null  float64
 11  Humidity       26800 non-null  float64
 12  Wind_speed     26800 non-null  float64
 13  Precipitation  26800 non-null  float64
dtypes: float64(12), object(2)
memory usage: 4.2+ MB


In [4]:
df['Time'] = pd.to_datetime(df['Time'], errors="coerce")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39561 entries, 0 to 39560
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Time           39561 non-null  datetime64[ns]
 1   AQI            39561 non-null  float64       
 2   PM2.5          39561 non-null  float64       
 3   PM10           39561 non-null  float64       
 4   CO             39561 non-null  float64       
 5   NO2            39561 non-null  float64       
 6   O3             39561 non-null  float64       
 7   SO2            39561 non-null  float64       
 8   NH3            39561 non-null  float64       
 9   Date           39561 non-null  object        
 10  Temp           26800 non-null  float64       
 11  Humidity       26800 non-null  float64       
 12  Wind_speed     26800 non-null  float64       
 13  Precipitation  26800 non-null  float64       
dtypes: datetime64[ns](1), float64(12), object(1)
memory usage: 4.2+ MB


In [6]:
fire_arch = pd.read_csv('fire_archive_J1V-C2_630824.csv')
data_2_nrt = pd.read_csv('fire_nrt_J1V-C2_630824.csv')

In [7]:
data_2 = pd.concat([fire_arch, data_2_nrt], ignore_index=True)
print(data_2.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 442 entries, 0 to 441
Data columns (total 15 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   latitude    442 non-null    float64
 1   longitude   442 non-null    float64
 2   brightness  442 non-null    float64
 3   scan        442 non-null    float64
 4   track       442 non-null    float64
 5   acq_date    442 non-null    object 
 6   acq_time    442 non-null    int64  
 7   satellite   442 non-null    object 
 8   instrument  442 non-null    object 
 9   confidence  442 non-null    object 
 10  version     442 non-null    object 
 11  bright_t31  442 non-null    float64
 12  frp         442 non-null    float64
 13  daynight    442 non-null    object 
 14  type        425 non-null    float64
dtypes: float64(8), int64(1), object(6)
memory usage: 51.9+ KB
None


In [8]:
### Step 4. Filter bounding box Hà Nội + lân cận

data_2_filtered = data_2[
    (data_2['latitude'] >= 20.5) & (data_2['latitude'] <= 21.5) &
    (data_2['longitude'] >= 105.3) & (data_2['longitude'] <= 106.1)
]
print("Số điểm cháy trong vùng:", len(data_2_filtered))


Số điểm cháy trong vùng: 442


In [12]:
data_2_filtered['acq_date'] = pd.to_datetime(data_2_filtered['acq_date'], errors="coerce")
data_2_filtered = data_2_filtered.dropna(subset=['acq_date'])
data_2_filtered['Date'] = data_2_filtered['acq_date'].dt.date

fire_count = data_2_filtered.groupby('Date').size().reset_index(name='fire_count')
print(fire_count.head())


         Date  fire_count
0  2020-11-29           1
1  2020-12-04           2
2  2020-12-05           2
3  2020-12-06           1
4  2020-12-08           2


In [10]:
df_merge = pd.merge(df, fire_count, how='left', on='Date')
df_merge['fire_count'] = df_merge['fire_count'].fillna(0)
df_merge

,Time,AQI,PM2.5,PM10,CO,NO2,O3,SO2,NH3,Date,Temp,Humidity,Wind_speed,Precipitation,fire_count
0,2020-11-25 08:00:00,5.0,270.20,295.95,2590.18,77.46,0.78,76.29,12.79,2020-11-25,22.37,90.1,1.6,0.01,0.0
1,2020-11-25 09:00:00,5.0,252.62,277.52,2670.29,77.46,3.44,75.34,13.68,2020-11-25,22.37,90.1,1.6,0.01,0.0
2,2020-11-25 10:00:00,5.0,218.28,240.00,2269.75,79.51,11.27,71.53,13.43,2020-11-25,22.37,90.1,1.6,0.01,0.0
3,2020-11-25 11:00:00,5.0,149.64,161.84,1321.79,61.01,51.50,51.98,7.41,2020-11-25,22.37,90.1,1.6,0.01,0.0
4,2020-11-25 12:00:00,5.0,122.29,130.78,1054.76,47.30,75.82,44.35,6.14,2020-11-25,22.37,90.1,1.6,0.01,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39556,2025-06-30 12:00:00,2.0,13.01,14.75,249.12,2.43,38.19,1.66,2.49,2025-06-30,NaN,NaN,NaN,NaN,0.0
39557,2025-06-30 13:00:00,2.0,13.90,15.36,229.19,1.93,43.03,1.53,2.38,2025-06-30,NaN,NaN,NaN,NaN,0.0
39558,2025-06-30 14:00:00,2.0,10.24,11.33,207.99,1.81,39.19,1.14,2.00,2025-06-30,NaN,NaN,NaN,NaN,0.0
39559,2025-06-30 15:00:00,1.0,7.38,8.34,201.18,2.08,33.95,0.99,1.86,2025-06-30,NaN,NaN,NaN,NaN,0.0


In [13]:
print(df_merge.isnull().sum())
df_merge = df_merge.drop_duplicates()


Time                 0
AQI                  0
PM2.5                0
PM10                 0
CO                   0
NO2                  0
O3                   0
SO2                  0
NH3                  0
Date                 0
Temp             12761
Humidity         12761
Wind_speed       12761
Precipitation    12761
fire_count           0
dtype: int64


In [16]:
d = pd.read_csv('data.csv')
d.info()
d.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39561 entries, 0 to 39560
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Time           39561 non-null  object 
 1   AQI            39561 non-null  float64
 2   PM2.5          39561 non-null  float64
 3   PM10           39561 non-null  float64
 4   CO             39561 non-null  float64
 5   NO2            39561 non-null  float64
 6   O3             39561 non-null  float64
 7   SO2            39561 non-null  float64
 8   NH3            39561 non-null  float64
 9   Date           39561 non-null  object 
 10  Temp           26800 non-null  float64
 11  Humidity       26800 non-null  float64
 12  Wind_speed     26800 non-null  float64
 13  Precipitation  26800 non-null  float64
 14  fire_count     39561 non-null  float64
dtypes: float64(13), object(2)
memory usage: 4.5+ MB


,AQI,PM2.5,PM10,CO,NO2,O3,SO2,NH3,Temp,Humidity,Wind_speed,Precipitation,fire_count
count,39561.000000,39561.000000,39561.000000,39561.00000,39561.000000,39561.000000,39561.000000,39561.000000,26800.000000,26800.000000,26800.000000,26800.000000,39561.0
mean,4.265244,109.965766,126.100224,1779.67638,47.118179,32.470684,38.398339,16.011014,23.169636,83.728543,1.726218,4.816287,0.0
std,1.020837,102.014142,125.035928,1649.57483,76.583574,72.320024,22.817834,17.885866,5.524550,6.331484,0.668806,9.850924,0.0
min,1.000000,0.000000,-9999.000000,117.52000,-9999.000000,-9999.000000,0.370000,0.000000,7.250000,58.100000,0.590000,0.000000,0.0
25%,4.000000,40.430000,49.900000,827.79000,30.500000,0.160000,24.800000,7.790000,19.080000,80.680000,1.240000,0.030000,0.0
50%,5.000000,75.720000,88.370000,1214.98000,42.840000,10.010000,33.380000,10.890000,24.400000,85.120000,1.600000,0.640000,0.0
75%,5.000000,143.070000,161.840000,2029.42000,59.630000,44.620000,47.210000,16.970000,27.930000,88.280000,2.090000,4.820000,0.0
max,5.000000,821.580000,926.830000,14312.74000,260.470000,583.650000,267.030000,263.450000,32.050000,95.500000,5.470000,112.050000,0.0


In [18]:
missing = df[['Temp', 'Humidity', 'Wind_speed', 'Precipitation']].isnull().sum()
print(missing)

Temp             12761
Humidity         12761
Wind_speed       12761
Precipitation    12761
dtype: int64


In [ ]:
# df_merge.to_csv('data.csv', index=False)
# print('Lưu data thành công!')

Lưu data thành công!


### Nhận xét 

#### **1. Thiếu dữ liệu Weather**
- Columns ảnh hưởng: `Temp`, `Humidity`, `Wind_speed`, `Precipitation`
- Mising: ~12,761 rows (~32% dataset)

#### **2. Placeholder values (-9999)**

- **Columns:** `PM10`, `NO2`, `O3`
- **Giá trị placeholder:** `-9999`

#### **3. AQI **

- **Mean AQI:** 4.26  
- **Median AQI:** 5.0 ➔ phần lớn observations AQI = 5 (**“Rất kém”**)

#### **4.Fire count**
- **Vấn đề:** Toàn bộ = 0


#### **5. Outliers / Extreme values chưa xử lý**

| **Cột** | **Max value** | **Nhận xét** |
|---|---|---|
CO | 14,312.74 | Rất cao, cần verify đơn vị API |
PM2.5 | 821.58 | Vượt ngưỡng WHO |
O3 | 583.65 | Có -9999 và giá trị lớn bất thường |

#### **6. Thiếu feature engineering time series**

- Hiện mới có: `Time`, `Date`
- **Thiếu:**
  - Hour, day_of_week, month, season
  - Lag features (VD: PM2.5 trước 1h, 3h)
  - Rolling mean, rolling std




####  **7. Đơn vị CO cần verify**

- **Mean CO:** ~1779 µg/m³  
- **Max CO:** 14,312.74 µg/m³  
